In [1]:
### NEEDS pyCloudCompare.py in same directory as this .ipynb file ###

### this code outputs a mesh representation of lichen-covered carvings.

### BRIEF OVERVIEW
### It takes input of RDLA lichen, turns it into RDLA with volume (represented 
### as a point cloud), but no meshing via distance mapping
### It then meshes the RDLA volume using ball pivoting for surface reconstruction
### It then performs dust accumulation to simulate the use of surface remote sensing
### such as laser scanning or photogrammetry on the RDLA mesh
### We sample the lowest z-value points, by placing a grid over the dust accumulation
### point cloud and calculating the index of the point in the dust accumulation with the
### smallest z-coordinate in each grid space.
### We then mesh these lowest z coordinate points by calculating normals for each point 
### and then using Poisson surface reconstruction. 
### We simplify this mesh into 1024 faces by using the quadratic edge collapse decimation
### algorithm.


import os
import pymeshlab as ml
import numpy as np
import open3d as o3d
import pyCloudCompare as cc
import matplotlib.pyplot as plt
import time
from time import strftime, localtime
from pathlib import Path

# Start timing
start_time = time.time()

# Set the input folder containing .obj meshes
#input_folder = r"C:\Users\gavin\OneDrive - University College London\ALL_CARVINGS_4_MESHNET\RDLA_non-carving_data_3.9.23"
input_folder = r"C:\Users\gavin\OneDrive - University College London\ALL_CARVINGS_4_MESHNET\RDLA_carving_mesh_only_7.9.23"
# Set the output folder where the processed meshes will be saved
output_folder = r"C:\Users\gavin\OneDrive - University College London\ALL_CARVINGS_4_MESHNET\junk"
# Set the seed mesh folder for mesh to cloud distance calculation
seed_folder = r"C:\Users\gavin\OneDrive - University College London\ALL_CARVINGS_4_MESHNET\carving_seed_mesh_4.9.23"
#distance map parameters:

# grid divison n x n x n
distanceMapGridstep = 0.0007
# Calculate the clearance value (e.g., 5 mm)
clearance = 0.01

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


def import_point_cloud_from_obj(obj_file_path):
    try:
        # Load the .obj file using open3d
        pcd = o3d.io.read_point_cloud(obj_file_path, format='ply')
        # Convert the point cloud data to a numpy array
        points = pcd.points

        # Convert numpy array to a list of tuples (x, y, z)
        point_cloud = [tuple(point) for point in points]

        return point_cloud
        #print(point_cloud)

    except Exception as e:
        print("Error importing point cloud:", e)
        return None

def divide_point_cloud_into_grid(point_cloud, grid_size=(45, 45)):
    # Get the bounding box of the point cloud
    min_bound = np.min(point_cloud, axis=0)
    max_bound = np.max(point_cloud, axis=0)
    
    # Calculate the size of each grid cell
    grid_size_x = (max_bound[0] - min_bound[0]) / grid_size[0]
    grid_size_y = (max_bound[1] - min_bound[1]) / grid_size[1]

    # Initialize an array to hold the points in each grid cell
    grid_cells = [[[] for _ in range(grid_size[1])] for _ in range(grid_size[0])]
    
    # Iterate through each point in the point cloud and find its grid cell
    for point in point_cloud:
        x_idx = int((point[0] - min_bound[0]) // grid_size_x)
        y_idx = int((point[1] - min_bound[1]) // grid_size_y)

        # Ensure the point is within the grid boundaries
        x_idx = min(grid_size[0] - 1, max(0, x_idx))
        y_idx = min(grid_size[1] - 1, max(0, y_idx))

        # Add the point to the corresponding grid cell
        grid_cells[x_idx][y_idx].append(point)

    return grid_cells

# Example usage:
# Replace "path/to/your_point_cloud.obj" with the actual path to your .obj file
# point_cloud = o3d.io.read_point_cloud("path/to/your_point_cloud.obj")
# if point_cloud is not None:
#     point_cloud_array = np.asarray(point_cloud.points)
#     grid_cells = divide_point_cloud_into_grid(point_cloud_array)

def calculate_minimum_in_grid_cells(grid_cells, grid_division=45):
    # Calculate the minimum value in each grid cell
    min_points = []
    for row in grid_cells:
        for cell in row:
            if cell:
                # Get the index of the point with the minimum z-coordinate value
                min_z_index = np.argmin([point[2] for point in cell])
                min_point = cell[min_z_index]
                min_points.append(min_point)
            else:
                # If a grid cell is empty, append a placeholder value (e.g., None)
                min_points.append(None)

    return min_points[:grid_division*grid_division]  # Return the first 1024 points (in case there are more than 1024 cells)

# Example usage:
# Assuming you have already imported and divided the point cloud into grid_cells
# min_values = calculate_minimum_in_grid_cells(grid_cells)

# MAXIMUM IS FOR CALCULATING THE HEIGHT OF THE RDLA LICHEN DISTANCE AWAY FROM THE SAMPLE MESH
# but we don't use it because we opt for calculating distance from samples mesh by point to mesh calculation
# using cloud compare
def calculate_maximum_in_grid_cells(grid_cells, grid_division=45):
    # Calculate the maximum value in each grid cell
    max_points = []
    for row in grid_cells:
        for cell in row:
            if cell:
                # Get the index of the point with the maximum z-coordinate value
                max_z_index = np.argmax([point[2] for point in cell])
                max_point = cell[max_z_index]
                max_points.append(max_point)
            else:
                # If a grid cell is empty, append a placeholder value (e.g., None)
                max_points.append(None)

    return max_points[:grid_division*grid_division]  # Return the first 1024 points (in case there are more than 1024 cells)

# Example usage:
# Assuming you have already imported and divided the point cloud into grid_cells
# max_values = calculate_maximum_in_grid_cells(grid_cells)


def save_points_to_ply_file(min_points, file_path):
    # Filter out the None elements from min_points
    valid_points = [point for point in min_points if point is not None]

    # Convert the list of points to a numpy array
    point_array = np.array(valid_points)

    # Create an Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_array)

    # Save the point cloud to a .ply file
    o3d.io.write_point_cloud(file_path, pcd)

    
#f = open('readme.txt', 'w') 
# Loop through all .obj files in the input folder
for file in sorted(Path(input_folder).iterdir(), key=os.path.getmtime):
    if file.parts[-1].endswith('596_RDLA_final_10242023-09-04_17-58step07min0575max1175.ply'): # .endswith(".csv"):
        
        # save file portion extracted for saving file name
        #filepart = file.parts[-1][5:9]
        filepart = file.parts[-1][0:4]
        ### DISTANCE MAPPING
        
        # import point cloud (uses open3d)
        #point_cloud = import_point_cloud_from_obj(input_folder + '\\' + file)
        np_ptcld = np.genfromtxt(input_folder + '\\' + file.parts[-1], delimiter=',')
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(np_ptcld)
        
        # Get the minimum and maximum coordinates of the point cloud
        min_bound = np.asarray(pcd.get_min_bound())
        max_bound = np.asarray(pcd.get_max_bound())
        
        # Adjust the minimum and maximum bounds to create the bounding cube with clearance
        min_bound -= clearance
        max_bound += clearance
        
        # Calculate the number of points needed in each dimension of the grid
        num_points_x = int(np.ceil((max_bound[0] - min_bound[0]) / distanceMapGridstep))
        num_points_y = int(np.ceil((max_bound[1] - min_bound[1]) / distanceMapGridstep))
        num_points_z = int(np.ceil((max_bound[2] - min_bound[2]) / distanceMapGridstep))
        
        # Generate the query points grid using numpy.mgrid with the calculated number of points
        grid_x, grid_y, grid_z = np.mgrid[min_bound[0]:max_bound[0]:num_points_x*1j,
                                          min_bound[1]:max_bound[1]:num_points_y*1j,
                                          min_bound[2]:max_bound[2]:num_points_z*1j]
        
        # Combine the grid points into a single array of shape [num_points, 3]
        query_points = np.column_stack((grid_x.ravel(), grid_y.ravel(), grid_z.ravel())).astype(np.float32)

        
        # Convert the array to a PointCloud object in Open3D
        query_point_cloud = o3d.geometry.PointCloud()
        query_point_cloud.points = o3d.utility.Vector3dVector(query_points.reshape(-1, 3))
        
        # Calculate distances from the point cloud to the query points
        distances = o3d.geometry.PointCloud.compute_point_cloud_distance(query_point_cloud, pcd)
        
        # Convert distances to a NumPy array for reshaping and visualization
        distances_np = np.asarray(distances)
        
        # Reshape the distances_np array to match the shape of the query grid
        query_shape = (len(grid_x), len(grid_x[0]), len(grid_x[0][0]))
        distances_reshaped = distances_np.reshape(query_shape)
        
        # Define the set distance you want to find points at (e.g., 1.5 mm)
        set_distance_min = 0.00085 - 0.0003
        set_distance_max = 0.00085 + 0.0003
        #set_distance_min = 0
        #set_distance_max = 10
        
        # Find the indices of the query points that are approximately 1.5 mm away from the point cloud
        indices_dist_range = np.asarray(np.logical_and(np.abs(distances_np) >= set_distance_min,
                                               np.abs(distances_np) <= set_distance_max)).nonzero()
        
        # Retrieve the coordinates of the points that are approximately 1.5 mm away from the point cloud
        points_dist_range = np.asarray(query_point_cloud.points)[indices_dist_range]

        save_points_to_ply_file(points_dist_range, output_folder + '\\' + filepart + 'RDLA_' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')

        #################################################################
        ### BALL PIVOTING
        # create meshlab object
        ball_pivot_mesh = ml.MeshSet()

        # fill meshlab object with points from the extracted distance map
        vertices = np.asarray(points_dist_range)
        mesh_converted = ml.Mesh(vertices)
        ball_pivot_mesh.add_mesh(mesh_converted, "ball_pivoted_mesh")

        # if point cloud simplification is needed
        #ball_pivot_mesh.apply_filter("generate_simplified_point_cloud", radius = ml.AbsoluteValue(0.0008))
        
        # Apply the ball pivot algorithm for meshing distance map points
        ball_pivot_mesh.apply_filter("generate_surface_reconstruction_ball_pivoting", ballradius = ml.AbsoluteValue(0.0009), creasethr = 150)

        # close any remaining holes from the ball-pivoting
        ball_pivot_mesh.apply_filter("meshing_close_holes")

        # file name for ball pivot mesh output
        ball_pivot_file_name = os.path.join(output_folder, filepart + 'RDLA_ball_recon' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        ball_pivot_mesh.save_current_mesh(ball_pivot_file_name)

        #################################################################
        # DUST ACCUMULATION

        # Apply the generate_dust_accumulation_point_cloud function
        ball_pivot_mesh.apply_filter("generate_dust_accumulation_point_cloud", dust_dir = [0,-0.70711,0.70711], nparticles = 1)

        dust_accumulation_file_name = os.path.join(output_folder, filepart + 'RDLA_dust' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        ball_pivot_mesh.save_current_mesh(dust_accumulation_file_name)
        
        
        #################################################################
        ### MIN Z GRID
        point_cloud = import_point_cloud_from_obj(dust_accumulation_file_name)
        
        grid_division = 50
        divided_ptcld = divide_point_cloud_into_grid(point_cloud, (grid_division,grid_division))
        final_cell = calculate_minimum_in_grid_cells(divided_ptcld, grid_division)
        #print(len(final_cell))

        min_z_file_name = os.path.join(output_folder, filepart + 'RDLA_lowest_z' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        save_points_to_ply_file(final_cell, min_z_file_name)
        
        #################################################################
        ### NORMALS CALCULATION

        # create meshlab object
        output_mesh = ml.MeshSet()

        # Filter out the None elements from min_points
        valid_points = [point for point in final_cell if point is not None]
        # Convert the list of points to a numpy array
        point_array = np.array(valid_points)
                
        # fill meshlab object with points from the extracted distance map
        vertices = np.asarray(point_array)
        mesh_converted = ml.Mesh(vertices)
        output_mesh.add_mesh(mesh_converted, "ball_pivoted_mesh")
        
        output_mesh.apply_filter("compute_normal_for_point_clouds", k = 6, flipflag = True, viewpos = [0, 0, 1])
        #################################################################
        ### SURFACE RECONSTUCTION SCREENED POISSON

        output_mesh.apply_filter("generate_surface_reconstruction_screened_poisson", scale = 1.0)
        
        #################################################################
        ### SIMPLIFICATION: QUADRATIC EDGE COLLAPSE DECIMATION: 1024
        output_mesh.apply_filter("meshing_decimation_quadric_edge_collapse", targetfacenum = 1024, qualitythr = 1.0, 
                                 preservenormal = True, preservetopology = True)
        
        final_1024_mesh = os.path.join(output_folder, filepart + 'RDLA_final_1024' + strftime('%Y-%m-%d_%H-%M', localtime(start_time)) 
                                + 'step' + str(distanceMapGridstep)[4:] + 'min' + str(set_distance_min)[4:]
                                + 'max' + str(set_distance_max)[4:] + '.ply')
        
        # save mesh using pymeshlab saving method
        output_mesh.save_current_mesh(final_1024_mesh)

        
        for file2 in os.listdir(seed_folder):
            if file2.endswith("1024_596_fixed.obj"):
                cli = cc.CloudCompareCLI()
                cmd = cli.new_command()
                cmd.silent()  # Disable console
                cmd.open(seed_folder + '\\' + file2)  # Read file
                cmd.open(final_1024_mesh)  # Read file
                cmd.c2m_dist()
                #cmd.cloud_export_format(cc.CLOUD_EXPORT_FORMAT.ASCII, extension="csv")
                #cmd.save_clouds('GAMMA_CALCULATE_OUTPUT\\' + file.parts[-1][:27] + '.csv')
                print(cmd)
                cmd.execute()
        
        
        
        ### 
        
        #break

# End timing
end_time = time.time()
execution_time = end_time - start_time

print(f"Execution time: {execution_time:.4f} seconds")

ImportError: cannot import name 'BaseResponse' from 'werkzeug.wrappers' (C:\Users\gavin\AppData\Roaming\Python\Python39\site-packages\werkzeug\wrappers\__init__.py)